In [ ]:
#Use this section to access a new dataset that needs to be used for the splitting and model
import pandas as pd
selected_df = pd.read_csv('/content/drive/MyDrive/sjsu nsf reu/edited_merged_files.csv')
selected_df.shape

(14415958, 10)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = selected_df.loc[:, ['StartTime', 'Dur', 'Proto', 'SrcAddr', 'DstAddr', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', 'Label']]
y = selected_df.loc[:, 'Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 420)

In [ ]:
#9
background_count = 0
normal_count = 0
botnet_count = 0

for x in X_train['Label']:
    if '1' in str(x): #Background
        background_count += 1
    if '2' in str(x): #Normal
        normal_count += 1
    if '3' in str(x): #Botnet
        botnet_count += 1

print(str(float(background_count/11532766)))
print(str(float(normal_count/11532766)))
print(str(float(botnet_count/11532766)))

botnet_ratio_b = (float(botnet_count/background_count))
botnet_ratio_n = (float(botnet_count/normal_count))


0.9741123681864351
0.015892804900402905
0.009994826913162029


In [ ]:
#14
filtered_df_1 = X_train.copy()

# Create a boolean mask to identify rows with "background" in the 'Label' column
mask_b = filtered_df_1['Label'].astype(str).str.contains('1')

# Identify the indices of the rows to drop
indices_to_drop_b = filtered_df_1.index[mask_b].tolist()[:(background_count-botnet_count)]
filtered_y = y_train.copy()

# Drop the specified rows from the DataFrame
filtered_df_1 = filtered_df_1.drop(indices_to_drop_b)
filtered_y = filtered_y.drop(indices_to_drop_b)

# Reset the index of the filtered DataFrame
filtered_df_1 = filtered_df_1.reset_index(drop=True)
filtered_y = filtered_y.reset_index(drop=True)

In [ ]:
#15
background_count = 0
normal_count = 0
botnet_count = 0

for x in filtered_df_1['Label']:
    if '1' in str(x): #Background
        background_count += 1
    if '2' in str(x): #Normal
        normal_count += 1
    if '3' in str(x): #Botnet
        botnet_count += 1

print(str(float(background_count/413824)))
print(str(float(normal_count/413824)))
print(str(float(botnet_count/413824)))

botnet_ratio_b = (float(botnet_count/background_count))
botnet_ratio_n = (float(botnet_count/normal_count))

0.2785435354160223
0.44291292916795544
0.2785435354160223


In [ ]:
#17
filtered_df = filtered_df_1.copy()

# Create a boolean mask to identify rows with "normal" in the 'Label' column
mask_n = filtered_df['Label'].astype(str).str.contains('2')

# Identify the indices of the rows to drop
indices_to_drop_n = filtered_df.index[mask_n].tolist()[:(normal_count-botnet_count)]
filtered_y_2 = filtered_y.copy()

# Drop the specified rows from the DataFrame
filtered_df = filtered_df.drop(indices_to_drop_n)
filtered_y_2 = filtered_y_2.drop(indices_to_drop_n)

# Reset the index of the filtered DataFrame
filtered_df = filtered_df.reset_index(drop=True)
filtered_y_2 = filtered_y_2.reset_index(drop=True)

In [ ]:
X_train = filtered_df

In [ ]:
y_train = filtered_y_2

In [ ]:
X_train = X_train.drop(columns=['Label'])
X_test = X_test.drop(columns=['Label'])

In [ ]:
X_train.shape

(345804, 9)

In [ ]:
X_test.shape

(2883192, 9)

In [ ]:
y_train.value_counts()

3    115268
2    115268
1    115268
Name: Label, dtype: int64

In [ ]:
y_test.value_counts()

1    2808812
2      45688
3      28692
Name: Label, dtype: int64

In [ ]:
y_test.shape

(2883192,)

In [ ]:
X_split_index = len(X_test) // 2
X_val = X_test[X_split_index:]
X_test = X_test[:X_split_index]

In [ ]:
y_split_index = len(y_test) // 2
y_val = y_test[y_split_index:]
y_test = y_test[:y_split_index]

# DNN


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.utils import to_categorical

# Assuming y_train is your original target data
# Perform one-hot encoding on y_train
y_train_encoded = to_categorical(y_train)

y_val_encoded = to_categorical(y_val)

# Repeat the same for y_test if needed
y_test_encoded = to_categorical(y_test)

In [ ]:
y_val_encoded.shape

(1441596, 4)

In [ ]:
y_train_encoded.shape

(345804, 4)

In [ ]:
X_val.shape

(1441596, 9)

In [ ]:
model = keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(9,)),
    keras.layers.Dense(100, activation='relu', input_shape=(9,)),
    keras.layers.Dense(100, activation='relu', input_shape=(9,)),
    keras.layers.Dense(100, activation='relu', input_shape=(9,)),
    keras.layers.Dense(100, activation='relu', input_shape=(9,)),
    keras.layers.Dense(4, activation='softmax')
])

#custom_optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_encoded, epochs=5, batch_size=100, validation_data=(X_val, y_val_encoded))

test_loss, test_acc = model.evaluate(X_test, y_test_encoded)

Epoch 1/5
3459/3459 [==============================] - 53s 15ms/step - loss: 2782.2678 - accuracy: 0.4515 - val_loss: 5.3742 - val_accuracy: 0.2674
Epoch 2/5
3459/3459 [==============================] - 31s 9ms/step - loss: 3.2291 - accuracy: 0.4449 - val_loss: 1.3750 - val_accuracy: 0.2175
Epoch 3/5
3459/3459 [==============================] - 31s 9ms/step - loss: 32.3624 - accuracy: 0.3723 - val_loss: 1.0272 - val_accuracy: 0.1355
Epoch 4/5
3459/3459 [==============================] - 31s 9ms/step - loss: 1.1061 - accuracy: 0.3638 - val_loss: 1.0699 - val_accuracy: 0.0789
Epoch 5/5
45050/45050 [==============================] - 74s 2ms/step - loss: 1.0704 - accuracy: 0.9743


In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_acc*100.))

Test loss: 1.07. Test accuracy: 97.43%
